In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [2]:
#create Spark session
spark = SparkSession.builder.appName('YelpML').getOrCreate()

#change configuration settings on Spark 
conf = spark.sparkContext._conf.setAll([('spark.executor.memory', '5g'), ('spark.app.name', 'Spark Updated Conf'), ('spark.executor.cores', '4'), ('spark.cores.max', '4'), ('spark.driver.memory','8g')])

#print spark configuration settings
#spark.sparkContext.getConf().getAll()

In [3]:
# Import Data
dataDir = "gs://msca-bdp-student-gcs/group2/yelp-datasample2"
business = spark.read.json(dataDir + "/sample_business")
checkin = spark.read.json(dataDir + "/sample_checkin")
review = spark.read.json(dataDir + "/sample_review")
tip = spark.read.json(dataDir + "/sample_tip")
user = spark.read.json(dataDir + "/sample_user")

23/08/04 20:30:24 WARN org.apache.spark.sql.catalyst.util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


## User EDA

In [5]:
user.take(1)

[Row(average_stars=4.5, compliment_cool=0, compliment_cute=0, compliment_funny=0, compliment_hot=0, compliment_list=0, compliment_more=0, compliment_note=0, compliment_photos=0, compliment_plain=0, compliment_profile=0, compliment_writer=0, cool=1, elite='', fans=0, friends='None', funny=1, name='Thomas', review_count=2, useful=2, user_id='DQ-2d41j1kJ4Wyf8ZOiTKg', yelping_since='2015-07-28 02:50:14')]

In [6]:
user_eda = user.drop('friends')

user_eda.cache()
user_eda.take(5)

[Row(average_stars=4.5, compliment_cool=0, compliment_cute=0, compliment_funny=0, compliment_hot=0, compliment_list=0, compliment_more=0, compliment_note=0, compliment_photos=0, compliment_plain=0, compliment_profile=0, compliment_writer=0, cool=1, elite='', fans=0, funny=1, name='Thomas', review_count=2, useful=2, user_id='DQ-2d41j1kJ4Wyf8ZOiTKg', yelping_since='2015-07-28 02:50:14'),
 Row(average_stars=3.63, compliment_cool=0, compliment_cute=0, compliment_funny=0, compliment_hot=0, compliment_list=0, compliment_more=0, compliment_note=0, compliment_photos=0, compliment_plain=0, compliment_profile=0, compliment_writer=0, cool=0, elite='', fans=0, funny=0, name='Rob', review_count=8, useful=2, user_id='FX3rCFA1YA0J9UBnorU5pw', yelping_since='2013-09-22 02:16:32'),
 Row(average_stars=3.0, compliment_cool=0, compliment_cute=0, compliment_funny=0, compliment_hot=0, compliment_list=0, compliment_more=0, compliment_note=0, compliment_photos=0, compliment_plain=0, compliment_profile=0, comp

In [8]:
from pyspark.sql.functions import isnan, when, count, col
user_eda.select([count(when(user_eda[c].isNull(), c)).alias(c) for c in user_eda.columns]).show()

+-------------+---------------+---------------+----------------+--------------+---------------+---------------+---------------+-----------------+----------------+------------------+-----------------+----+-----+----+-----+----+------------+------+-------+-------------+
|average_stars|compliment_cool|compliment_cute|compliment_funny|compliment_hot|compliment_list|compliment_more|compliment_note|compliment_photos|compliment_plain|compliment_profile|compliment_writer|cool|elite|fans|funny|name|review_count|useful|user_id|yelping_since|
+-------------+---------------+---------------+----------------+--------------+---------------+---------------+---------------+-----------------+----------------+------------------+-----------------+----+-----+----+-----+----+------------+------+-------+-------------+
|            0|              0|              0|               0|             0|              0|              0|              0|                0|               0|                 0|            

In [9]:
user.count()

20006